# Face Recognition

## Face Recognition Pipeline Steps

1. Locate and extract faces from each image
2. Identify facial features in each image
3. Use position to align faces to match pose template
4. Use a neural network to create a set of measurements that represent that face
    - Encode faces using a trained neural network
    - Face encoding = measurements
5. Check Euclidean distance between face encodings (measurements)